In [1]:
import pandas as pd
import numpy as np
import yaml
from yaml.loader import SafeLoader
import os
import warnings
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

warnings.filterwarnings('ignore')

In [2]:
# Cambiamos el directorio de trabajo a la carpeta models
os.chdir('../../')

In [3]:
print("Directorio: {0}".format(os.getcwd()))

Directorio: /Users/mohamed.rios/Projects/desafio_etermax/models


# Leemos archivo de parámetros

In [4]:
with open('resources.yaml') as f:
    resources = yaml.load(f, Loader=SafeLoader)
    params = resources.get('preprocess_params')
    models_path = resources.get('artifact_path')

# Leemos los datasets para entrenar

In [5]:
x_train = pd.read_csv(params['x_train_path'])
x_test = pd.read_csv(params['x_test_path'])
y_train = pd.read_csv(params['y_train_path'])
y_test = pd.read_csv(params['y_test_path'])

# Pipelines de modelos a entrenar

In [6]:
lr = LinearRegression()
xgb = GradientBoostingRegressor()
models = [(lr, 'lr'), (xgb, 'xgb')]

In [7]:
for m in models:
    m[0].fit(x_train, y_train['revenue'])

# Guardo el binario de los modelos

In [8]:
for m in models:
    with open(models_path + '/' + m[1],'wb') as outfile:
        pickle.dump(m[0], outfile)